In [13]:
!pip install mutagen
!pip install transforms3d
import tensorflow as tf
from PIL import Image, ImageDraw
from google.colab import drive
import os
import matplotlib.pyplot as plt
import sys
import importlib
import shutil
import csv
import pandas as pd
import random
import cProfile
import pstats
import numpy as np
drive.mount('/content/drive')
# !rm -rf subject_txt
# !rm -rf all_asfamc
!unzip /content/drive/My\ Drive/Colab\ Notebooks/Pose\ Priors/data/allasfamc.zip
!unzip /content/drive/My\ Drive/Colab\ Notebooks/Pose\ Priors/data/subject_txt.zip
import amc_parser as a
import contextlib

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  /content/drive/My Drive/Colab Notebooks/Pose Priors/data/allasfamc.zip
   creating: all_asfamc/
   creating: all_asfamc/subjects/
   creating: all_asfamc/subjects/01/
  inflating: all_asfamc/subjects/01/01.asf  
  inflating: all_asfamc/subjects/01/01_01.amc  
  inflating: all_asfamc/subjects/01/01_02.amc  
  inflating: all_asfamc/subjects/01/01_03.amc  
  inflating: all_asfamc/subjects/01/01_04.amc  
  inflating: all_asfamc/subjects/01/01_05.amc  
  inflating: all_asfamc/subjects/01/01_06.amc  
  inflating: all_asfamc/subjects/01/01_07.amc  
  inflating: all_asfamc/subjects/01/01_08.amc  
  inflating: all_asfamc/subjects/01/01_09.amc  
  inflating: all_asfamc/subjects/01/01_10.amc  
  inflating: all_asfamc/subjects/01/01_11.amc  
  inflating: all_asfamc/subjects/01/01_12.amc  
  inflating: all_asfamc/subjects/01/01_13.amc  
  inflating: all_asfamc/s

In [ ]:
def genData(subject, makeImgs, jointlist):
  sub = "{:0>2d}".format(subject)
  basepath = '/content/drive/My Drive/Colab Notebooks/Pose Priors/'
  subpath = '/content/all_asfamc/subjects/'+sub+'/'
  paths = checkPaths(sub, makeImgs, basepath)
  # try: 
  if makeImgs: csvPath, imgPath, gifPath = paths 
  else: csvPath = paths
  trials = sorted(os.listdir(subpath)) #ind 0 will be asf, rest amc
  for ind in range(len(trials)):
    trial = trials[ind]
    if ind == 0:
      asf_path = subpath+trial
      joints = a.parse_asf(asf_path)
      continue #don't do any amc processing
    trial_num = "{:0>2d}".format(ind) #for amc files
    amc_path = subpath+trial
    motions = a.parse_amc(amc_path) #get all motions in amc file
    # output = pd.DataFrame(columns=jointlist)
    output = np.zeros((len(motions), len(jointlist)*3))
    if makeImgs and ind == 1: #save first trial for indicated subjects
        im = Image.new('RGB', (200,200))
        imgpath = imgPath+trial_num+'/'
        os.mkdir(imgpath)
    for idx in range(len(motions)): #make all images in trial
        mot = "{:0>6d}".format(idx)
        joints['root'].set_motion(motions[idx])
        # print(joints['root'].coordinate)
        # lambda l: [item for sublist in l for item in sublist]
        flattened = [val[0]
                     for sublist in [joints[joint].coordinate 
                     for joint in jointlist]
                     for val in sublist] 
        if makeImgs and ind == 1:
          joints['root'].draw(imgpath+mot+'.png')
        # output = output.append(coordinates, ignore_index=True)
        # output[idx] = np.squeeze(np.asarray(coordinates), axis=2)
        output[idx] = np.asarray(flattened)
    
    # output.to_csv(csvPath+trial_num+'.csv')
    np.savetxt(csvPath+trial_num+".csv", output, delimiter=",")
    # print("saved")
    if makeImgs and ind == 1:
      gifpath = gifPath+trial_num+'.gif'
      makeGif(gifpath, imgpath)
  # except: 
  #   print(sub+" had an error.")
  #   print(sys.exc_info()[0])

In [ ]:
def checkPaths(sub, makeImgs, basepath):
  csvPath = basepath+'data/'+sub+'/'
  imgPath = basepath+'imgs/'+sub+'/'
  gifPath = basepath+'gifs/'+sub+'/'
  if os.path.exists(imgPath):
    shutil.rmtree(imgPath)
  if os.path.exists(gifPath):
    shutil.rmtree(gifPath)
  if os.path.exists(csvPath):
    shutil.rmtree(csvPath)
  os.mkdir(csvPath)
  if makeImgs: #check if gif/imgs directories exist - delete if they do
    os.mkdir(imgPath)
    os.mkdir(gifPath)
    return csvPath, imgPath, gifPath
  else: return csvPath

In [ ]:
makeGif('/content/drive/My Drive/Colab Notebooks/Pose Priors/gifs/17/01.gif', '/content/drive/My Drive/Colab Notebooks/Pose Priors/imgs/17/01/')

NameError: ignored

In [ ]:
def makeGif(gifpath, imgpath):
  figs = sorted(os.listdir(imgpath))
  frames = []
  for i in figs:
      new_frame = Image.open(imgpath+i)
      frames.append(new_frame)
  # Save into a GIF file that loops forever
  frames[0].save(gifpath, format='GIF',
          append_images=frames[1:],
            save_all=True, duration=len(frames)/120) #same as 120 fps but not sure how accurate fps setting is

In [2]:
def makeSequence(subject, amc, desc, jointlist, xyz_header):

  sub = "{:0>2d}".format(subject)
  trial_num = amc[amc.find('_')+1:amc.find('.')]
  trial = int(trial_num)
  amcpath = '/content/all_asfamc/subjects/'+sub+'/'+amc
  asfpath = '/content/all_asfamc/subjects/'+sub+'/'+sub+'.asf'
  f = open(asfpath, "r")
  asf = f.read()
  test_motions = a.parse_amc(amcpath)
  angles = test_motions[0].keys()
  angle_header = []
  endings = ['-x', '-y', '-z', '-x2', '-y2', '-z2']
  for angle in angles:
    for i in range(len(test_motions[0][angle])):
      angle_header.append(angle+endings[i])
  angle_header_encoded = [x.encode('utf-8') for x in angle_header]
  xyz_header_encoded = [x.encode('utf-8') for x in xyz_header]

  id = str(hash(sub+'/'+trial_num))+'/'+sub+'/'+trial_num

  ex = tf.train.SequenceExample()
  ex.context.feature["example/id"].bytes_list.value.append(id.encode("utf8"))
  ex.context.feature["example/asf"].bytes_list.value.append(asf.encode("utf8"))
  ex.context.feature["example/angle_header"].bytes_list.value.extend(angle_header_encoded)
  ex.context.feature["example/xyz_header"].bytes_list.value.extend(xyz_header_encoded)
  ex.context.feature["example/description"].bytes_list.value.append(desc.encode("utf8"))

  timestamp = [int(i/120*1000000) for i in range(len(test_motions))]
  anglelist = [[motion[l] for l in angles] for motion in test_motions]
  angles = [[item for sublist in r for item in sublist] for r in anglelist]

  headers_nested = sequenceHelper(subject, trial, jointlist)
  for t in timestamp:
    ex.feature_lists.feature_list["ALL_JOINT_ANGLES/feature/timestamp"].feature.add().int64_list.value.append(t)
    ex.feature_lists.feature_list["ALL_JOINT_XYZ/feature/timestamp"].feature.add().int64_list.value.append(t)
  for angle in angles:
    ex.feature_lists.feature_list["ALL_JOINT_ANGLES/feature/floats"].feature.add().float_list.value[:] = angle
  for header in headers_nested:
    ex.feature_lists.feature_list["ALL_JOINT_XYZ/feature/floats"].feature.add().float_list.value[:] = header

  return ex 

In [3]:
def sequenceHelper(subject, t, jointlist):
  sub = "{:0>2d}".format(subject)
  trial = "{:0>2d}".format(t) #for amc files
  subpath = '/content/all_asfamc/subjects/'+sub+'/'
  asf_path = subpath+sub+'.asf'
  amc_path = subpath+sub+'_'+trial+'.amc'
  joints = a.parse_asf(asf_path)
  motions = a.parse_amc(amc_path) #get all motions in amc file
  output = np.zeros((len(motions), len(jointlist)*3))
  for idx in range(len(motions)): #make all images in trial
    mot = "{:0>6d}".format(idx)
    joints['root'].set_motion(motions[idx])
    flattened = [val[0]
        for sublist in [joints[joint].coordinate 
        for joint in jointlist]
        for val in sublist] 
    output[idx] = np.asarray(flattened)
  return output

In [4]:
def getDescription(sub, trial_amc):
  filename = "/content/subject_txt/subject" + str(sub) + ".txt" 
  d = {}
  with open(filename) as fin:
      next(fin) 
      next(fin)
      next(fin)
      for line in fin:
        try:
          ind = line.find('.avi')+5
          words = line.split()
          amc = [word for word in words if "amc" in word][0]
          d[amc] = line[ind:]
        except: pass
      return d[trial_amc]

In [5]:
def _generate_single_example(subject, trial_amc):
    """For each row in the annotation CSV, generates the corresponding metadata.
    Args:
      subject: subject number (integer)
      trial_amc: amc filename 
    Yields:
      Each tf.SequenceExample of metadata
    """

    xyz_header = []
    jointlist = ['root', 'lhipjoint', 'lfemur', 'ltibia', 'lfoot', 'ltoes', 'rhipjoint', 'rfemur', 'rtibia', 'rfoot', 'rtoes', 'lowerback', 'upperback', 'thorax', 'lowerneck', 'upperneck', 'head', 'lclavicle', 'lhumerus', 'lradius', 'lwrist', 'lhand', 'lfingers', 'lthumb', 'rclavicle', 'rhumerus', 'rradius', 'rwrist', 'rhand', 'rfingers', 'rthumb']
    for joint in jointlist:
      xyz_header.append(joint+'-x')
      xyz_header.append(joint+'-y')
      xyz_header.append(joint+'-z')
    description = getDescription(subject, trial_amc)
    metadata = makeSequence(subject, trial_amc, description, jointlist, xyz_header)     
    return metadata

In [14]:
def generate_splits():
    SPLITS = {}
    add = ["train", "test"]
    SPLITS["train"] = []
    SPLITS["test"] = []
    path = '/content/all_asfamc/subjects/'
    for subjectfolder in os.listdir(path)[:2]:
      amcs = sorted(os.listdir(path+subjectfolder))[1:]
      for ind in range(len(amcs)):
        SPLITS[add[ind % 2]].append(amcs[ind]) 
    return SPLITS

In [15]:
class MotionDataset(object):
  """Generates and loads a motion capture data set."""

  def __init__(self, path_to_data):
    if not path_to_data:
      raise ValueError("You must supply the path to the data directory.")
    self.path_to_data = path_to_data


  def as_dataset(self,
                 split,
                 shuffle=False,
                 repeat=False,
                 serialized_prefetch_size=32,
                 decoded_prefetch_size=32):
    """Returns the dataset as a tf.data.Dataset.
    Args:
      split: either "train" or "test"
      shuffle: if true, shuffles both files and examples.
      repeat: if true, repeats the data set forever.
      serialized_prefetch_size: the buffer size for reading from disk.
      decoded_prefetch_size: the buffer size after decoding.
    Returns:
      A tf.data.Dataset object with the following structure: {
        "images": uint8 tensor, shape [time, height, width, channels]
        "labels": one hot encoded label tensor, shape [2]
        "id": a unique string id for each example, shape []
      }
    """

    def parse_fn(sequence_example):
      #replace keys with strings
      context_features = {
          "example/id": tf.io.FixedLenFeature(shape=(), dtype=tf.string) ,
          "example/asf": 
              tf.io.FixedLenFeature(shape=(), dtype=tf.string),
          "example/angle_header": 
              tf.io.FixedLenFeature(shape=(62,), dtype=tf.string),
          "example/xyz_header": 
              tf.io.FixedLenFeature(shape=(93,), dtype=tf.string),
          "example/description": 
              tf.io.FixedLenFeature(shape=(), dtype=tf.string),
      }

      sequence_features = {
          "ALL_JOINT_ANGLES/feature/timestamp": 
              tf.io.FixedLenSequenceFeature(shape=(), dtype=tf.int64),
          "ALL_JOINT_XYZ/feature/timestamp": 
              tf.io.FixedLenSequenceFeature(shape=(), dtype=tf.int64),  
          "ALL_JOINT_ANGLES/feature/floats": 
              tf.io.FixedLenSequenceFeature(shape=(62,), dtype=tf.float32),
          "ALL_JOINT_XYZ/feature/floats": 
              tf.io.FixedLenSequenceFeature(shape=(93,), dtype=tf.float32),
      }
      
      parsed_context, parsed_sequence = tf.io.parse_single_sequence_example(
          sequence_example, context_features, sequence_features)
      return parsed_sequence

    SPLITS = generate_splits(self.path_to_data)
    if split not in SPLITS:
      raise ValueError("split '%s' is unknown." % split)
    all_shards = tf.io.gfile.glob(
        os.path.join(self.path_to_data, TF_RECORD_PATTERN % split + "-*-of-*"))
    if shuffle:
      random.shuffle(all_shards)
    all_shards_dataset = tf.data.Dataset.from_tensor_slices(all_shards)
    cycle_length = min(32, len(all_shards))
    dataset = all_shards_dataset.interleave( ##
        tf.data.TFRecordDataset,
            cycle_length=cycle_length,
            block_length=1)
    dataset = dataset.prefetch(serialized_prefetch_size)
    if shuffle:
      dataset = dataset.shuffle(serialized_prefetch_size)
    if repeat:
      dataset = dataset.repeat()
    dataset = dataset.map(parse_fn)
    dataset = dataset.prefetch(decoded_prefetch_size)
    return dataset

  def generate_examples(self):
    """Downloads data and generates sharded TFRecords.
    Downloads the data files, generates metadata, and processes the metadata
    with MediaPipe to produce tf.SequenceExamples for training. The resulting
    files can be read with as_dataset(). After running this function the
    original data files can be deleted.
    Args:
      path_to_mediapipe_binary: Path to the compiled binary for the BUILD target
        mediapipe/examples/desktop/demo:media_sequence_demo.
      path_to_graph_directory: Path to the directory with MediaPipe graphs in
        mediapipe/graphs/media_sequence/.
    """
    SPLITS = generate_splits()
    for split in SPLITS:
      all_metadata = []
      for trial_amc in SPLITS[split]:
        #get subject and trial_amc, append to all_metadata
        sub = int(trial_amc[:trial_amc.find('_')])
        metadata = (sub, trial_amc)
        all_metadata.append(metadata)
      random.seed(47)
      random.shuffle(all_metadata) #get pairs in split
      shard_names = [self._indexed_shard(split, i) for i in range(NUM_SHARDS)]
      writers = [tf.io.TFRecordWriter(shard_name) for shard_name in shard_names]
      with _close_on_exit(writers) as writers:
        for i, (sub, trial_amc) in enumerate(all_metadata):
          try:
            seq_ex = _generate_single_example(sub, trial_amc)
            writers[i % len(writers)].write(seq_ex.SerializeToString())
          except: 
            print("Something went wrong.")

  def _indexed_shard(self, split, index):
    """Constructs a sharded filename."""
    return os.path.join(
        self.path_to_data,
        TF_RECORD_PATTERN % split + "-%05d-of-%05d" % (index, NUM_SHARDS))
    
@contextlib.contextmanager
def _close_on_exit(writers):
  """Call close on all writers on exit."""
  try:
    yield writers
  finally:
    for writer in writers:
      writer.close()

In [9]:
NUM_SHARDS = 10 # ~200MB/file
TF_RECORD_PATTERN = "mocap_dataset_%s_tfrecord"

In [10]:
mocap = MotionDataset("/content/drive/My Drive/Colab Notebooks/Pose Priors")
# mocap.generate_examples()

In [28]:
train = mocap.as_dataset("train")
test = mocap.as_dataset("test")
# xyz = next(iter(dataset))
# xyz['ALL_JOINT_XYZ/feature/floats'] #processing operations -> parse_fn 

In [19]:
#@title Import { display-mode: "form" }


import numpy as np

import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()

import tensorflow_datasets as tfds
import tensorflow_probability as tfp


tfk = tf.keras
tfkl = tf.keras.layers
tfpl = tfp.layers
tfd = tfp.distributions

In [29]:
# datasets, datasets_info = tfds.load(name='mnist',
#                                     with_info=True,
#                                     as_supervised=False)

def _preprocess(sample):
  xyz = tf.cast(sample['ALL_JOINT_XYZ/feature/floats'], tf.float32) / 255.  # Scale to unit interval.
  xyz = xyz < tf.random.uniform(tf.shape(xyz))   # Randomly binarize.
  return xyz, xyz

train_dataset = (train
                 .map(_preprocess)
                 .batch(256)
                 .prefetch(tf.data.experimental.AUTOTUNE)
                 .shuffle(int(10e3)))
eval_dataset = (test
                .map(_preprocess)
                .batch(256)
                .prefetch(tf.data.experimental.AUTOTUNE))

In [30]:
xyz = next(iter(train))
input_shape = xyz['ALL_JOINT_XYZ/feature/floats'].shape
encoded_size = 16
base_depth = 32

# train_dataset

In [32]:
prior = tfd.Independent(tfd.Normal(loc=tf.zeros(encoded_size), scale=1),
                        reinterpreted_batch_ndims=1)

In [33]:
encoder = tfk.Sequential([
    tfkl.InputLayer(input_shape=input_shape),
    tfkl.Lambda(lambda x: tf.cast(x, tf.float32) - 0.5),
    tfkl.Dense(base_depth, activation=tf.nn.leaky_relu),
    # tfkl.Conv2D(base_depth, 5, strides=1,
    #             padding='same', activation=tf.nn.leaky_relu),
    tfkl.Dense(2 * base_depth, activation=tf.nn.leaky_relu),
    # tfkl.Conv2D(base_depth, 5, strides=2,
    #             padding='same', activation=tf.nn.leaky_relu),
    # tfkl.Conv2D(2 * base_depth, 5, strides=1,
    #             padding='same', activation=tf.nn.leaky_relu),
    # tfkl.Conv2D(2 * base_depth, 5, strides=2,
    #             padding='same', activation=tf.nn.leaky_relu),
    # tfkl.Conv2D(4 * encoded_size, 7, strides=1,
    #             padding='valid', activation=tf.nn.leaky_relu),
    tfkl.Dense(4 * encoded_size, activation=tf.nn.leaky_relu),
    tfkl.Flatten(),
    tfkl.Dense(tfpl.MultivariateNormalTriL.params_size(encoded_size),
               activation=None),
    tfpl.MultivariateNormalTriL(
        encoded_size,
        activity_regularizer=tfpl.KLDivergenceRegularizer(prior)),
])

Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.


In [35]:
decoder = tfk.Sequential([
    tfkl.InputLayer(input_shape=[encoded_size]),
    tfkl.Reshape([1, 1, encoded_size]),
    # tfkl.Conv2DTranspose(2 * base_depth, 7, strides=1,
    #                      padding='valid', activation=tf.nn.leaky_relu),
    # tfkl.Conv2DTranspose(2 * base_depth, 5, strides=1,
    #                      padding='same', activation=tf.nn.leaky_relu),
    # tfkl.Conv2DTranspose(2 * base_depth, 5, strides=2,
    #                      padding='same', activation=tf.nn.leaky_relu),
    # tfkl.Conv2DTranspose(base_depth, 5, strides=1,
    #                      padding='same', activation=tf.nn.leaky_relu),
    # tfkl.Conv2DTranspose(base_depth, 5, strides=2,
    #                      padding='same', activation=tf.nn.leaky_relu),
    # tfkl.Conv2DTranspose(base_depth, 5, strides=1,
                        #  padding='same', activation=tf.nn.leaky_relu),
    # tfkl.Conv2D(filters=1, kernel_size=5, strides=1,
    #             padding='same', activation=None),
    tfkl.Dense(2 * base_depth, activation=tf.nn.leaky_relu),
    tfkl.Dense(base_depth, activation=tf.nn.leaky_relu),
    tfkl.Flatten(),
    tfpl.IndependentNormal(input_shape,tfd.Distribution.mean),
])

In [36]:
vae = tfk.Model(inputs=encoder.inputs,
                outputs=decoder(encoder.outputs[0]))

In [38]:
negloglik = lambda x, rv_x: -rv_x.log_prob(x)

vae.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-3),
            loss=negloglik)

_ = vae.fit(train_dataset,
            epochs=15,
            validation_data=test_dataset)

Epoch 1/15


TypeError: ignored

In [ ]:
# We'll just examine ten random digits.
x = next(iter(eval_dataset))[0][:10]
xhat = vae(x)
assert isinstance(xhat, tfd.Distribution)